# Real-Time Semantic Segmentation - Google Colab Deployment

This notebook deploys the real-time segmentation server on Google Colab with GPU acceleration.

## Setup Instructions

1. **Enable GPU**: Go to `Runtime` → `Change runtime type` → Select `GPU`
2. **Get ngrok token**: Sign up at https://ngrok.com and get your auth token
3. **Run cells in order**: Execute each cell sequentially
4. **Access app**: Use the ngrok URL provided in the output

## Note
- Colab sessions timeout after 12 hours or 90 minutes idle
- GPU allocation is not guaranteed (T4/P100/V100 depending on availability)
- Save models to Google Drive for persistence across sessions

## 1. Check GPU Availability

In [ ]:
!nvidia-smi

## 2. Clone Repository

In [ ]:
# Clone your repository (replace with your repo URL)
!git clone https://github.com/yourusername/RealTimeSeg.git
%cd RealTimeSeg

## 3. Install Dependencies

In [ ]:
# Install PyTorch with CUDA support
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install other requirements
!pip install -r backend/requirements.txt

# Install ngrok for tunneling
!pip install pyngrok

## 4. Download and Cache Models (Optional)

Pre-download models to speed up startup. Models will be cached for the session.

In [ ]:
import torch
import torchvision.models.segmentation as models
from transformers import SegformerForSemanticSegmentation

print("Downloading models...")

# Download DeepLabV3 models
print("1/3 Downloading DeepLabV3-MobileNetV3...")
_ = models.deeplabv3_mobilenet_v3_large(pretrained=True)

print("2/3 Downloading DeepLabV3-ResNet50...")
_ = models.deeplabv3_resnet50(pretrained=True)

print("3/3 Downloading SegFormer-B3...")
_ = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b3-finetuned-ade-512-512"
)

print("✓ All models downloaded and cached")

## 5. Setup ngrok Tunnel

**Important**: Replace `YOUR_NGROK_TOKEN` with your actual ngrok auth token from https://ngrok.com

In [ ]:
from pyngrok import ngrok, conf
import os

# Set your ngrok auth token here
NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # Replace with your token

# Configure ngrok
conf.get_default().auth_token = NGROK_TOKEN

# Create tunnel
public_url = ngrok.connect(8000)
print(f"\n{'='*60}")
print(f"🌐 PUBLIC URL: {public_url}")
print(f"{'='*60}\n")
print(f"Open this URL in your browser to access the app!")
print(f"\nNote: This URL is valid for this Colab session only.")

## 6. Start Server

This will start the FastAPI server. The cell will keep running - don't stop it!

**Access your app**: Open the ngrok URL from the previous cell in your browser.

In [ ]:
import sys
sys.path.append('backend')

# Set environment to use GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Start the server
!cd backend && python app.py

## 7. Keep Session Alive (Optional)

Run this in a separate cell to prevent Colab from disconnecting due to inactivity.

**Note**: This is optional and should be used responsibly.

In [ ]:
import time
from datetime import datetime

print("Keep-alive started. Press 'Stop' button to end.")
print("This will print a message every 60 seconds.\n")

try:
    while True:
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"[{current_time}] Session active...")
        time.sleep(60)
except KeyboardInterrupt:
    print("\nKeep-alive stopped.")

## Troubleshooting

### GPU Not Available
- Go to `Runtime` → `Change runtime type` → Select `GPU`
- Restart runtime and run cells again

### ngrok Connection Failed
- Check that your ngrok token is correct
- Free tier has connection limits (40 connections/min)
- Try regenerating the tunnel (run ngrok cell again)

### Server Won't Start
- Check that all dependencies installed successfully
- Make sure you're in the correct directory
- Review error messages in cell output

### Models Loading Slowly
- First run downloads models (~2-3 GB)
- Subsequent runs use cached models (faster)
- Consider mounting Google Drive for persistent cache

## Performance Tips

1. **Start with Balanced Mode**: Good trade-off between speed and accuracy
2. **Monitor GPU Memory**: Check `nvidia-smi` if experiencing issues
3. **Reduce Resolution**: If laggy, lower webcam resolution in browser
4. **Close Other Tabs**: Browser resources affect webcam capture performance